In [1]:
from tkinter import *
import cv2
import os
import math
import imutils
import numpy as np
from tkinter import filedialog
from PIL import Image,ImageTk
import time
import threading
from tkinter import ttk

class MainWindow():
    
    def __init__(self, main):
        
        super().__init__()
        
        self.main=main
        self.coordinate=[]
        self.fp=[]
        self.sp=[]
        self.rstimg=None
        self.firstroi=[]
        self.secroi=[]
        self.l=[]
        self.imgwidth=0
        self.imgheight=0
        self.btnpressed=''
        self.src=""
        self.dest=""
        self.imglist=[]
        self.isprocessed=False
        self.source="source"
        self.clicked_point=[]
        
        self.main.geometry('1300x650')
        
        self.title=Label(root, font=('Helvatical bold',14), text="OMR Pre-processing Application", width=120, bg='darkgray')
        self.title.place(x=0, y=0)
        self.imgnum = 0
        
        self.frame = Frame(root,bg='darkgray',width=300,height=640)
        self.frame.place(x=0, y=28)
        
        self.sourcebtn = Button(main, text = 'Source', bd = '5', command = self.getSource)
        self.sourcebtn.place(x=10, y=40) 
        
        self.slabel=Label(root, text="No source directory choosen !", bg='yellow', width=30)
        self.slabel.place(x=70, y=40)
        
        self.destinationBtn = Button(main, text = 'Destination', bd = '5', command = self.destination)
        self.destinationBtn.place(x=10, y=80)
        
        self.dlabel=Label(root, text="NO destination directory choosen !", bg='yellow', width=26)
        self.dlabel.place(x=100, y=80)
        
        self.sheetlabel=Label(root, text="Select Sheet Type", bg='gray')
        self.sheetlabel.place(x=10, y=160)
        
        label = Label( root, text='Play Ground', relief=RAISED )
        options=['pd','marks','mcq']
        self.sheettype = StringVar()
        self.sheettype.set( "pd" )
        self.choosesheettype = OptionMenu( root , self.sheettype , *options )
        self.choosesheettype.place(x=130, y=160) 
        
        self.roilabel=Label(root, text="-------------------- Select ROI --------------------------", bg='lightgray')
        self.roilabel.place(x=0, y=240)
        
        self.firstRoiBtn = Button(main, text = 'First ROI', bd = '5', command = self.firstRoi)
        self.firstRoiBtn.place(x=20, y=270) 
        
        self.secondRoiBtn = Button(main, text = 'Second ROI', bd = '5', command = self.secondRoi)
        self.secondRoiBtn.place(x=100, y=270) 
        
        self.processAllBtn = Button(main, text = 'Process All', bd = '5', command = self.processAll)
        self.processAllBtn.place(x=190, y=270)
        
        self.roilabel=Label(root, text="----------------------------------------------------------", bg='lightgray')
        self.roilabel.place(x=0, y=310)
        
        self.processOneByOneBtn = Button(main, text = 'Process Manually', bd = '5',  state=DISABLED, command = self.processonebyone)
        self.processOneByOneBtn.place(x=10, y=380)
        
        self.resetBtn = Button(main, text = 'Reset', bd = '5', command = self.resetimage)
        self.resetBtn.place(x=130, y=380)
        
        self.saveBtn = Button(main, text = 'save', bd = '5', command = self.saveimage)
        self.saveBtn.place(x=200, y=380)
        

        self.loadAllbtn = Button(main, text = 'load', bd = '5', command = self.loadAll)
        self.loadAllbtn.place(x=10, y=120)
        
        self.processedSheetBtn = Button(main, text = 'Processed Sheet', bd = '5', command = self.processedSheet)
        self.processedSheetBtn.place(x=60, y=120) 
        
        self.notProcessedSheetBtn = Button(main, text = 'Not Processed Sheet', bd = '5', command = self.notProcessedSheet)
        self.notProcessedSheetBtn.place(x=170, y=120) 
        
        self.processedSheetBtn.config(state=DISABLED)
        self.notProcessedSheetBtn.config(state=DISABLED)
        
        self.exitbtn = Button(main, text = '               EXIT             ', bd = '5', command = root.destroy)
        self.exitbtn.place(x=70, y=450)  
        
        label.place_forget()
        self.l1=Label(root, font=('Helvatical bold',73), text='OMR\n Pre - Processing', bg='lightgray', fg='white')
        self.l1.place(x=400,y=200)
        
        
    def getSource(self):
        self.src= filedialog.askdirectory()
        if self.src != "":
            self.slabel.config(text= self.src, bg='green')
    
    def loadAll(self):
        self.source='source'
        self.imgnum=0
        self.imglist=os.listdir(self.src)
        self.stotal=len(self.imglist)
        self.title.config(text="Total Sheet:-"+"  "+str(self.stotal))
        self.l1.place_forget()
        self.display()
       
    def on_canvas_configure(self, event):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))

    def on_mousewheel(self, event):
        if event.delta:
            self.canvas.yview_scroll(-1 * (event.delta // 120), "units")
            
         # Calculate the new scrollregion based on the current view and scrolling direction
        x0, y0, x1, y1 = self.canvas.bbox("all")  # Get the bounding box of all items on the canvas
        if event.delta > 0:  # Scroll up
            self.canvas.yview_scroll(-1, "units")  # Scroll one unit up
        else:  # Scroll down
            self.canvas.yview_scroll(1, "units")  # Scroll one unit down
        x0, y0, x1, y1 = self.canvas.bbox("all")  # Update the bounding box
        self.canvas.configure(scrollregion=(x0, y0, x1, y1))
        

    def display(self):
       
        if self.source=='source':
            self.img=Image.open(self.src+'//'+self.imglist[self.imgnum])
        elif self.source=='processed':
            self.img=Image.open(self.dest+'//processed//'+self.imglist[self.imgnum])
        elif self.source=='not_processed':
            self.img=Image.open(self.dest+'//not_processed//'+self.imglist[self.imgnum])
            self.img1=self.img
            
        self.org=self.img
        self.img=np.array(self.img)
        self.imgwidth=self.img.shape[1]
        self.imgheight=self.img.shape[0]
#         self.img=Image.fromarray(self.img)
        

        self.main_win_title=self.imglist[0]
        self.main.title(self.main_win_title)
    
        self.frame1 = Frame(self.main, width=740, height=590, bg="green")
        self.frame1.place(x=430, y=30)
    
        self.canvas = Canvas(self.frame1, width=740, height=590, bg='white')
        hbar = Scrollbar(self.frame1, orient=HORIZONTAL)
        hbar.pack(side=BOTTOM, fill=X)
        hbar.config(command=self.canvas.xview)
        vbar = Scrollbar(self.frame1, orient=VERTICAL)
        vbar.pack(side=RIGHT, fill=Y)
        vbar.config(command=self.canvas.yview)
        self.canvas.config(xscrollcommand=hbar.set, yscrollcommand=vbar.set)
        self.canvas.pack(side=RIGHT, expand=True, fill=BOTH)
        
        self.cimg=ImageTk.PhotoImage(image=self.org)
        self.canvasimage=self.canvas.create_image(0, 0, anchor=NW, image=self.cimg)
        
        self.canvas.itemconfig(self.canvasimage, image=self.cimg )
        self.canvas.bind("<Configure>", self.on_canvas_configure)
        self.canvas.bind("<MouseWheel>", self.on_mousewheel)
       
        self.nextbtn = Button(self.main, text = 'NEXT', bd = '5', command = self.nextimage)
        self.prebtn = Button(self.main, text = 'PREVIOUS', bd = '5', command = self.previousimage)
        self.nextbtn.place(x=1200, y=400) 
        self.prebtn.place(x=310, y=400)         
        
        
    def destination(self):
        self.dest = filedialog.askdirectory()
        if self.dest != "":
            self.dlabel.config(text= self.dest, bg='green')
            path = self.dest+"//not_processed"
            path1 = self.dest+"//processed"
            try: 
                os.mkdir(path) 
            except OSError as error: 
                print(error)
                
            try: 
                os.mkdir(path1) 
            except OSError as error: 
                print(error)
        else:
            self.dlabel.config(text= "NO destination directory choosen !", bg='yellow')
    
# /////////////////\\\\\\\\\\\\\\\\\\\\\\////\\\\\\////\\\\\\\////\\\//

    def on_canvas_click(self,event):
        x = event.x
        y = event.y
        self.clicked_point.append([x,y])
#         print(self.clicked_point)
        
        if (len(self.clicked_point) == 2):
            angle = math.degrees(math.atan2(self.clicked_point[1][1] - self.clicked_point[0][1], self.clicked_point[1][0] - self.clicked_point[0][0]))
            self.img = Image.open(self.dest+"//not_processed//" + self.imglist[self.imgnum])
            self.img=np.array(self.img)
            self.org=self.img
            self.img=imutils.rotate(self.img, angle)
            self.cimg = ImageTk.PhotoImage(Image.fromarray(self.img))            

            self.canvas.itemconfig(self.canvasimage, image=self.cimg )
            
#             self.nextimage()

    def processonebyone(self):
        self.source='not_processed'
        self.imglist=os.listdir(self.dest+"//not_processed")
        self.imgnum=0
        self.display()
        self.canvas.bind("<Button-1>", self.on_canvas_click)

    def create_progress_frame(self):
        self.progress_frame = Frame(self.main, width=300, height=100, bg='lightgray')
        self.progress_frame.place(x=550, y=320)

        self.progress_label = Label(self.progress_frame, text="Processing...", bg='lightgray')
        self.progress_label.place(x=20, y=10)
        self.progress_bar = ttk.Progressbar(self.progress_frame, mode='determinate', length=260)
        self.progress_bar.place(x=20, y=30)
        
        self.close_button = Button(self.progress_frame, text='Close', command=self.cancel_process)
        self.close_button.place(x=140, y=60)
      
    def update_progress_bar(self, value):
        self.progress_bar['value'] = value
        self.progress_label.config(text=f"Processing {self.pic} images...")
        self.main.update_idletasks() 
        
    
    def cancel_process(self):
        self.progress_frame.destroy()
    
        
    def processAll(self):
        # Create a progress bar
        self.create_progress_frame()
        # Start the processing in a separate thread
        self.process_thread = threading.Thread(target=self.process_images)
        self.process_thread.start()
                
                
    def process_images(self):
        if(self.src==''):
            self.alertmsg("Please select source directory")
            pass
        if(self.dest==''):
            self.alertmsg("Please select destination directory")
            pass
            
        not_processed=self.dest+"//not_processed//"
        processedimg=[]
        notprocessedimg=[]
        self.pic=0
        for img in self.imglist:
            processed=''
            image=cv2.imread(self.src+'//'+img) 
            try:
                print(img)
                fp,sp = MainWindow.get_coordinate(image.copy(), self.firstroi, self.secroi, self.sheettype.get())
                print("before rotation:- ", fp, sp)
                angle = math.degrees(math.atan2(sp[1] - fp[1], sp[0] - fp[0]))
                print(angle)

                processed=imutils.rotate(image, angle)
                cv2.imwrite(self.dest+'//processed//'+img, processed)
                processedimg.append(img)
            except:
                print("co-ordinate not found")
                cv2.imwrite(not_processed+'//'+img, image)
                notprocessedimg.append(img)
                
            self.pic += 1
            progress = (self.pic / len(self.imglist)) * 100
            self.update_progress_bar(progress)
            
        self.isprocessed=True
        self.title.config(text="Processed:- "+str(len(processedimg))+"    "+"Not Processed:- "+str(len(notprocessedimg)))
        if len(processedimg) != 0:
            self.processedSheetBtn.config(state=NORMAL)
        if len(notprocessedimg) != 0:
            self.notProcessedSheetBtn.config(state=NORMAL)
            self.processOneByOneBtn.config(state=NORMAL)
        
        print("Total processed pd image:", len(processedimg))
        print("Not processed pd image:", len(notprocessedimg))
        self.alertmsg('Total processed pd image: '+str(len(processedimg))+'\nNot processed pd image: '+ str(len(notprocessedimg)))
        
        
        
    @staticmethod
    def get_coordinate(image, froi, sroi, sheettype):  
        print(sheettype)
        grayInput = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #     _, binaryImage = cv2.threshold(grayInput, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, binaryImage = cv2.threshold(grayInput, 100, 255, cv2.THRESH_BINARY)

        gausimg=cv2.GaussianBlur(binaryImage, (3,3),cv2.BORDER_DEFAULT)
        cannyImage = cv2.Canny(gausimg, threshold1=180, threshold2=255, edges=1)

#         cv2.imshow("window", cannyImage)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        
        fp=[]
        sp=[]
        
        contours, hierarchy = cv2.findContours(cannyImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            x,y,w,h=cv2.boundingRect(cnt)
            if (x>froi[0][0] and x < froi[0][2] and y > froi[0][1] and y < froi[0][3]):
                area=cv2.contourArea(cnt)
                print(area)
                if(sheettype=='pd' or sheettype=='marks'):
                    if(area > 380 and area < 600):
                        image=cv2.rectangle(image,(x,y),(x+w, y+h),(0,255,0),2)
                        fp=[x,y]
                elif(sheettype=='mcq'):
                    if(area > 40 and area < 300):
                        image=cv2.rectangle(image,(x,y),(x+w, y+h),(0,255,0),2)
                        fp=[x,y]


            if (x > sroi[0][0] and x < sroi[0][2] and y > sroi[0][1] and y < sroi[0][3]):
                area=cv2.contourArea(cnt)
                if(sheettype=='pd' or sheettype=='marks'):
                    if(area > 380 and area < 600):
                        image=cv2.rectangle(image,(x,y),(x+w, y+h),(0,255,0),2)
                        sp=[x,y]
                elif(sheettype=='mcq'):
                    if(area > 40 and area < 300):
                        image=cv2.rectangle(image,(x,y),(x+w, y+h),(0,255,0),2)
                        sp=[x,y]
                    
        return fp,sp
        
        
    def firstRoi(self):
        self.btnpressed='firstroi'
        self.canvas.bind("<ButtonPress-1>", self.start_point) 
        self.canvas.bind("<ButtonRelease-1>",self.end_point)
        self.firstroi.append([x_start, y_start])
        self.firstroi.append([self.l[-1][0], self.l[-1][1]])
        self.l.clear()

    def secondRoi(self):
        self.btnpressed='secondroi'
        self.canvas.bind("<ButtonPress-1>", self.start_point) 
        self.canvas.bind("<ButtonRelease-1>",self.end_point)
        self.secroi.append([x_start, y_start])
        self.secroi.append([self.l[-1][0],self.l[-1][1]])
        
        
    def start_point(self, event):
        global x_start, y_start
#         x_start = event.x
#         y_start = event.y
        x_start, y_start = int(self.canvas.canvasx(event.x)), int(self.canvas.canvasy(event.y))
    
        
    def end_point(self, event):
        global x_end, y_end
#         x_end = event.x
#         y_end = event.y
        x_end, y_end = int(self.canvas.canvasx(event.x)), int(self.canvas.canvasy(event.y))
        
        
        self.canvas.delete("reactangle")
        self.l.append([x_end, y_end])
        self.canvas.create_rectangle(x_start, y_start,self.l[-1][0], self.l[-1][1], outline="red", tag='reactangle')
        
        if(self.btnpressed=='firstroi'):
            self.firstroi.clear()
            self.firstroi.append([x_start, y_start,self.l[-1][0], self.l[-1][1]])
           
        elif(self.btnpressed=='secondroi'):
            self.secroi.clear()
            self.secroi.append([x_start, y_start,self.l[-1][0], self.l[-1][1]])
          
        
    def nextimage(self):
        self.imgnum += 1
        # return to first image
        if self.imgnum == len(self.imglist):
            self.imgnum = 0
        
        self.main_win_title=self.imglist[self.imgnum]
        self.main.title(self.main_win_title)
       
        if self.source=='source':
            self.img=Image.open(self.src+'//'+self.imglist[self.imgnum])
        elif self.source=='processed':
            self.img=Image.open(self.dest+'//processed//'+self.imglist[self.imgnum])
        elif self.source=='not_processed':
            self.img=Image.open(self.dest+'//not_processed//'+self.imglist[self.imgnum])

        self.rstimg=self.img
        self.cimg = ImageTk.PhotoImage(self.img)
        # change image
        self.canvas.itemconfig(self.canvasimage, image=self.cimg )
    
    def previousimage(self):
        if self.imgnum > 0:
            self.imgnum -= 1
        elif self.imgnum == 0:
            self.imgnum = len(self.imglist)-1
       
        self.main_win_title=self.imglist[self.imgnum]
        self.main.title(self.main_win_title)
        
        
        if self.source=='source':
            self.img=Image.open(self.src+'//'+self.imglist[self.imgnum])
        elif self.source=='processed':
            self.img=Image.open(self.dest+'//processed//'+self.imglist[self.imgnum])
        elif self.source=='not_processed':
            self.img=Image.open(self.dest+'//not_processed//'+self.imglist[self.imgnum])
        
        self.rstimg=self.img
        self.cimg = ImageTk.PhotoImage(self.img)
        # change image
        self.canvas.itemconfig(self.canvasimage, image=self.cimg )
        
    def saveimage(self):
#         self.rimage = self.img
#         self.img.save(self.dest+"//processed"+ self.imglist[self.imgnum])
        cv2.imwrite(self.dest+"//processed//"+self.imglist[self.imgnum], Image.fromarray(np.array(self.img)))
        os.remove(self.dest+"//not_processed//"+self.imglist[self.imgnum])
        self.imglist.pop(self.imgnum)
        self.clicked_point=[]
        self.alertmsg("Image saved successfully !!")
        self.nextimage()
        
    def alertmsg(self, msg):  
        top = Toplevel(root) 
        top.geometry("300x150")
        label = Label(top, text = msg)
        button = Button(top, text = "Close", command = top.destroy)
        button.place(x=150, y=100)
        label.place(x=60, y=50)  
        top.mainloop()  
    
    def resetimage(self):
        self.clicked_point=[]
        self.cimg=ImageTk.PhotoImage(image=self.img1)
        self.canvas.itemconfig(self.canvasimage, image=self.cimg )
        
    def processedSheet(self):
        if(self.isprocessed):
#             self.canvas.destroy()
            self.source="processed"
            self.imglist=os.listdir(self.dest+"//processed")
            self.title.config(text="Processed:- "+str(len(self.imglist)))
            self.imgnum = 0
            if(len(self.imglist) !=0 ):
                self.display()
            else:
                self.alertmsg("Image not processed.")
        else:
            self.alertmsg("Image not processed yet.")
    
    def notProcessedSheet(self):
        if(self.isprocessed==True):
#             self.canvas.destroy()
            self.source="not_processed"
            self.imglist=os.listdir(self.dest+"//not_processed")
            self.title.config(text="Not Processed:- "+str(len(self.imglist)))
            self.imgnum = 0
            if(len(self.imglist) !=0 ):
                self.display()
            else:
                self.alertmsg("Image not processed.")
        else:
            self.alertmsg("Image not processed yet.")
            
    

if __name__ == "__main__":

    root = Tk()
    MainWindow(root)
    root.title("OMR Pre - Processing Application")
    root.resizable(width=False, height=False)
    root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\anaconda3\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-1-a6c036210210>", line 365, in firstRoi
    self.firstroi.append([x_start, y_start])
NameError: name 'x_start' is not defined


420001.jpg
pd
583.5
before rotation:-  [114, 84] [414, 85]
0.19098522435968746
420002.jpg
pd
525.0
before rotation:-  [108, 96] [409, 94]
-0.38069725142676386
420003.jpg
pd
570.0
before rotation:-  [107, 85] [407, 85]
0.0
420004.jpg
pd
570.0
before rotation:-  [117, 90] [417, 92]
0.3819662047290255
420005.jpg
pd
572.0
before rotation:-  [118, 96] [419, 98]
0.38069725142676386
420006.jpg
pd
584.5
before rotation:-  [117, 100] [418, 101]
0.19035072662975377
420007.jpg
pd
553.0
before rotation:-  [115, 100] [415, 101]
0.19098522435968746
420008.jpg
pd
554.5
before rotation:-  [113, 86] [413, 86]
0.0
420009.jpg
pd
569.5
before rotation:-  [116, 101] [417, 101]
0.0
420010.jpg
pd
554.0
before rotation:-  [121, 90] [420, 91]
0.19162396617427935
420011.jpg
pd
570.0
before rotation:-  [116, 97] [417, 97]
0.0
420012.jpg
pd
553.0
before rotation:-  [121, 85] [421, 87]
0.3819662047290255
420013.jpg
pd
580.5
before rotation:-  [114, 86] [414, 87]
0.19098522435968746
420014.jpg
pd
570.5
before rotat

Exception in thread Thread-6:
Traceback (most recent call last):
  File "d:\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "d:\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-a6c036210210>", line 309, in process_images
  File "<ipython-input-1-a6c036210210>", line 464, in alertmsg
  File "d:\anaconda3\lib\tkinter\__init__.py", line 1420, in mainloop
    self.tk.mainloop(n)
RuntimeError: Calling Tcl from different apartment
